In [62]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from joblib import dump

import warnings
warnings.filterwarnings('ignore')

In [63]:
data =  pd.read_csv('Sentiment.csv')
data

id               candidate  candidate_confidence relevant_yn  \
0          1  No candidate mentioned                1.0000         yes   
1          2            Scott Walker                1.0000         yes   
2          3  No candidate mentioned                1.0000         yes   
3          4  No candidate mentioned                1.0000         yes   
4          5            Donald Trump                1.0000         yes   
...      ...                     ...                   ...         ...   
13866  13867  No candidate mentioned                1.0000         yes   
13867  13868           Mike Huckabee                0.9611         yes   
13868  13869                Ted Cruz                1.0000         yes   
13869  13870            Donald Trump                1.0000         yes   
13870  13871                Ted Cruz                0.9242         yes   

       relevant_yn_confidence sentiment  sentiment_confidence  \
0                      1.0000   Neutral                0.6578   
1                      1.0000  Positive                0.6333   
2                      1.0000   Neutral                0.6629   
3                      1.0000  Positive                1.0000   
4                      1.0000  Positive                0.7045   
...                       ...       ...                   ...   
13866                  1.0000  Negative                0.7991   
13867                  1.0000  Positive                0.7302   
13868                  1.0000  Positive                0.8051   
13869                  1.0000  Negative                1.0000   
13870                  0.9614  Positive                0.9614   

                             subject_matter  subject_matter_confidence  \
0                         None of the above                     1.0000   
1                         None of the above                     1.0000   
2                         None of the above                     0.6629   
3                         None of the above                     0.7039   
4                         None of the above                     1.0000   
...                                     ...                        ...   
13866                              Abortion                     0.6014   
13867                     None of the above                     0.9229   
13868                     None of the above                     0.9647   
13869  Women's Issues (not abortion though)                     0.9202   
13870                     None of the above                     0.9242   

               candidate_gold  ... relevant_yn_gold retweet_count  \
0                         NaN  ...              NaN             5   
1                         NaN  ...              NaN            26   
2                         NaN  ...              NaN            27   
3                         NaN  ...              NaN           138   
4                         NaN  ...              NaN           156   
...                       ...  ...              ...           ...   
13866  No candidate mentioned  ...              yes             7   
13867           Mike Huckabee  ...              yes             1   
13868                Ted Cruz  ...              yes            67   
13869            Donald Trump  ...              yes           149   
13870                Ted Cruz  ...              yes            65   

          sentiment_gold                             subject_matter_gold  \
0                    NaN                                             NaN   
1                    NaN                                             NaN   
2                    NaN                                             NaN   
3                    NaN                                             NaN   
4                    NaN                                             NaN   
...                  ...                                             ...   
13866           Negative  Abortion\nWomen's Issues (not abortion though)   
13867         

In [64]:
data = data[['text', 'sentiment']]
data

text sentiment
0      RT @NancyLeeGrahn: How did everyone feel about...   Neutral
1      RT @ScottWalker: Didn't catch the full #GOPdeb...  Positive
2      RT @TJMShow: No mention of Tamir Rice and the ...   Neutral
3      RT @RobGeorge: That Carly Fiorina is trending ...  Positive
4      RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  Positive
...                                                  ...       ...
13866  RT @cappy_yarbrough: Love to see men who will ...  Negative
13867  RT @georgehenryw: Who thought Huckabee exceede...  Positive
13868  RT @Lrihendry: #TedCruz As President, I will a...  Positive
13869  RT @JRehling: #GOPDebate Donald Trump says tha...  Negative
13870  RT @Lrihendry: #TedCruz headed into the Presid...  Positive

[13871 rows x 2 columns]

In [65]:
data = data[data['sentiment']!= 'Neutral']
data

text sentiment
1      RT @ScottWalker: Didn't catch the full #GOPdeb...  Positive
3      RT @RobGeorge: That Carly Fiorina is trending ...  Positive
4      RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  Positive
5      RT @GregAbbott_TX: @TedCruz: "On my first day ...  Positive
6      RT @warriorwoman91: I liked her and was happy ...  Negative
...                                                  ...       ...
13866  RT @cappy_yarbrough: Love to see men who will ...  Negative
13867  RT @georgehenryw: Who thought Huckabee exceede...  Positive
13868  RT @Lrihendry: #TedCruz As President, I will a...  Positive
13869  RT @JRehling: #GOPDebate Donald Trump says tha...  Negative
13870  RT @Lrihendry: #TedCruz headed into the Presid...  Positive

[10729 rows x 2 columns]

In [66]:
for val in data['text']:
    print(val)

RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…
RT @RobGeorge: That Carly Fiorina is trending -- hours after HER debate -- above any of the men in just-completed #GOPdebate says she's on …
RT @DanScavino: #GOPDebate w/ @realDonaldTrump delivered the highest ratings in the history of presidential debates. #Trump2016 http://t.co…
RT @GregAbbott_TX: @TedCruz: "On my first day I will rescind every illegal executive action taken by Barack Obama." #GOPDebate @FoxNews
RT @warriorwoman91: I liked her and was happy when I heard she was going to be the moderator. Not anymore. #GOPDebate @megynkelly  https://…
Deer in the headlights RT @lizzwinstead: Ben Carson, may be the only brain surgeon who has performed a lobotomy on himself. #GOPDebate
RT @NancyOsborne180: Last night's debate proved it! #GOPDebate #BATsAsk @BadassTeachersA #TBATs  https://t.co/G2gGjY1bJD
@JGreenDC @realDonaldTrump In all fairness #

In [67]:
pattern = r"(RT\s@[\w_]+|#\w+|@\w+|http\S+|[^a-zA-Z\s])"

In [68]:
ps = PorterStemmer()
lmt = WordNetLemmatizer()

In [69]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

def clean_data(data):
    texts = []
    sentiments = []
    for index, row in data.iterrows():
        sentence = re.sub(pattern, '', row.text)
        words = [lmt.lemmatize(word.lower()) 
                 for word in sentence.split() if word.lower() not in stopwords.words('english')]
        texts.append(' '.join(words))
        sentiments.append(row.sentiment)
    return texts, sentiments

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BETSY\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BETSY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BETSY\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [70]:
data['text'], data['sentiment'] = clean_data(data)
data

text sentiment
1      didnt catch full last night scott best line se...  Positive
3      carly fiorina trending hour debate men justcom...  Positive
4      w delivered highest rating history presidentia...  Positive
5      first day rescind every illegal executive acti...  Positive
6              liked happy heard going moderator anymore  Negative
...                                                  ...       ...
13866       love see men never faced pregnancy talk body  Negative
13867              thought huckabee exceeded expectation  Positive
13868             president always tell truth said would  Positive
13869  donald trump say doesnt time political correct...  Negative
13870                  headed presidential debate go ted  Positive

[10729 rows x 2 columns]

In [71]:
analyzer = SentimentIntensityAnalyzer()

# Applying VADER to each row in your dataset to add sentiment score features
data['vader_compound'] = data['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
data['vader_pos'] = data['text'].apply(lambda x: analyzer.polarity_scores(x)['pos'])
data['vader_neg'] = data['text'].apply(lambda x: analyzer.polarity_scores(x)['neg'])

In [72]:
encode = LabelEncoder()
data['sentiment'] = encode.fit_transform(data['sentiment'])
data

text  sentiment  \
1      didnt catch full last night scott best line se...          1   
3      carly fiorina trending hour debate men justcom...          1   
4      w delivered highest rating history presidentia...          1   
5      first day rescind every illegal executive acti...          1   
6              liked happy heard going moderator anymore          0   
...                                                  ...        ...   
13866       love see men never faced pregnancy talk body          0   
13867              thought huckabee exceeded expectation          1   
13868             president always tell truth said would          1   
13869  donald trump say doesnt time political correct...          0   
13870                  headed presidential debate go ted          1   

       vader_compound  vader_pos  vader_neg  
1              0.6369      0.344      0.000  
3              0.0000      0.000      0.000  
4              0.0000      0.000      0.000  
5             -0.5574      0.000      0.286  
6              0.7579      0.619      0.000  
...               ...        ...        ...  
13866          0.6369      0.375      0.000  
13867          0.0000      0.000      0.000  
13868          0.3182      0.315      0.000  
13869          0.4939      0.225      0.000  
13870          0.0000      0.000      0.000  

[10729 rows x 5 columns]

### Initialize TfidfVectorizer

In [73]:
# Step 1: Vectorize the data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=10000, min_df=2, max_df=0.9)  # Limit to 10k features for efficiency
X = tfidf_vectorizer.fit_transform(data['text'])
print(X)

  (0, 603)	0.36206030882279855
  (0, 3698)	0.20241445241831835
  (0, 6645)	0.3386357853512096
  (0, 3906)	0.3256762759519123
  (0, 595)	0.26932924958703314
  (0, 6602)	0.2958297461131808
  (0, 4513)	0.19468924154882583
  (0, 3695)	0.19713649718197068
  (0, 2583)	0.35798015566542585
  (0, 1077)	0.42486846558915364
  (0, 1762)	0.24843168642935506
  (1, 1032)	0.3394048057041115
  (1, 6909)	0.3737237493071801
  (1, 6555)	0.24653037269972314
  (1, 4174)	0.28602127152447543
  (1, 1606)	0.18642938480368948
  (1, 3178)	0.3519161817160729
  (1, 8109)	0.49385545082092197
  (1, 2397)	0.3172600327485231
  (1, 1031)	0.3152800270930015
  (2, 3116)	0.29252320811946353
  (2, 5856)	0.29252320811946353
  (2, 3096)	0.29252320811946353
  (2, 1705)	0.29252320811946353
  (2, 5328)	0.21366102755121733
  :	:
  (10727, 4951)	0.17474869332303852
  (10727, 1855)	0.12407880430121046
  (10727, 1889)	0.12141298246346591
  (10727, 1882)	0.11693886339547027
  (10727, 898)	0.16056441397078902
  (10727, 2310)	0.1707265

In [74]:
# Step 2: Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

### Logistic regression

In [75]:
# Step 3: Train Logistic Regression
lr_model = LogisticRegression(class_weight={0: 1, 1: 1.5}, C= 1, max_iter=200)  # Increase max iterations for convergence
lr_model.fit(x_train, y_train)

LogisticRegression(C=1, class_weight={0: 1, 1: 1.5}, max_iter=200)

In [76]:
y_pred = lr_model.predict(x_test)
print(f"Accuracy: {lr_model.score(x_test, y_test)}")

Accuracy: 0.8555452003727866


In [77]:
# Accuracy
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))

Accuracy: 0.8555

Classification Report:
              precision    recall  f1-score   support

    Negative       0.87      0.96      0.91      1713
    Positive       0.74      0.44      0.55       433

    accuracy                           0.86      2146
   macro avg       0.80      0.70      0.73      2146
weighted avg       0.84      0.86      0.84      2146



In [78]:
cv_scores = cross_val_score(lr_model, x_train, y_train, cv=5)
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV accuracy: {cv_scores.mean()}")

Cross-Validation Scores: [0.84682586 0.85497962 0.83692487 0.84731935 0.85722611]
Mean CV accuracy: 0.8486551596336105


### Support Vector Machine

In [79]:
svm_model = SVC(class_weight={0:1, 1:1.5}, kernel='rbf', C=1, gamma='scale')
svm_model.fit(x_train, y_train)

SVC(C=1, class_weight={0: 1, 1: 1.5})

In [80]:
y_pred_svm = svm_model.predict(x_test)
print(f"Accuracy: {svm_model.score(x_test, y_test)}")

Accuracy: 0.8569431500465983


In [81]:
# Accuracy
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm, target_names=['Negative', 'Positive']))

Accuracy: 0.8569

Classification Report:
              precision    recall  f1-score   support

    Negative       0.88      0.95      0.91      1713
    Positive       0.72      0.47      0.57       433

    accuracy                           0.86      2146
   macro avg       0.80      0.71      0.74      2146
weighted avg       0.85      0.86      0.85      2146



In [82]:
cv_scores = cross_val_score(svm_model, x_train, y_train, cv=5)
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV accuracy: {cv_scores.mean()}")

Cross-Validation Scores: [0.84857309 0.86022132 0.83576005 0.8502331  0.8537296 ]
Mean CV accuracy: 0.8497034318816498


### Logistic Regression and SVM using SMOTE

In [83]:
# Initialize SMOTE
smote = SMOTE(sampling_strategy=0.75, random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

In [84]:
# Initialize and train Logistic Regression
lr_model1 = LogisticRegression(class_weight='balanced', C= 1, max_iter=200, random_state=42)
lr_model1.fit(x_train_resampled, y_train_resampled)

# Predict and evaluate
print("Logistic Regression Results with SMOTE:")
y_pred_lr = lr_model1.predict(x_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr, target_names=['Negative', 'Positive']))


Logistic Regression Results with SMOTE:
Accuracy: 0.8094

Classification Report:
              precision    recall  f1-score   support

    Negative       0.91      0.84      0.88      1713
    Positive       0.52      0.67      0.59       433

    accuracy                           0.81      2146
   macro avg       0.72      0.76      0.73      2146
weighted avg       0.83      0.81      0.82      2146



In [85]:
# Initialize and train SVM
svm_model1 = SVC(class_weight='balanced', kernel='rbf', C=1, gamma= 'scale', random_state=42)
svm_model1.fit(x_train_resampled, y_train_resampled)

# Predict and evaluate
print("SVM Results with SMOTE:")
y_pred_svm1 = svm_model1.predict(x_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm1):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm1, target_names=['Negative', 'Positive']))


SVM Results with SMOTE:
Accuracy: 0.8509

Classification Report:
              precision    recall  f1-score   support

    Negative       0.87      0.95      0.91      1713
    Positive       0.71      0.45      0.55       433

    accuracy                           0.85      2146
   macro avg       0.79      0.70      0.73      2146
weighted avg       0.84      0.85      0.84      2146



### Making prediction on unseen data using the best model

In [86]:
# Predicting on new data
new_data = ["Amazing experience from start to finish. Highly recommend to everyone"]  
new_data_vectorized = tfidf_vectorizer.transform(new_data)  

In [87]:
predictions = svm_model1.predict(new_data_vectorized)
for prediction in predictions:
    sentiment = "Positive" if prediction == 0 else "Negative"
    print("Prediction:", sentiment)

Prediction: Positive


### Saving the best model and the vectorizer

In [89]:
dump(svm_model1, 'best_model.joblib')
dump(tfidf_vectorizer, 'vectorizer.joblib')
print("Model saved as best_model.joblib & TfidfVectorizer saved as vectorizer.joblib")

Model saved as best_model.joblib & TfidfVectorizer saved as vectorizer.joblib
